**Install FastText to vm.**

In [1]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.1.zip
!unzip v0.9.1.zip
%cd fastText-0.9.1
!make

--2020-02-05 04:56:38--  https://github.com/facebookresearch/fastText/archive/v0.9.1.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.1 [following]
--2020-02-05 04:56:44--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.1
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.1.zip’

v0.9.1.zip              [  <=>               ]   4.13M  12.4MB/s    in 0.3s    

2020-02-05 04:56:44 (12.4 MB/s) - ‘v0.9.1.zip’ saved [4327207]

Archive:  v0.9.1.zip
b5b7d307274ce00ef52198fbc692ed3bd11d9856
   creating: fastText-0.9.1/
   creating: fastText-0.9.1/.circleci/
  inflating: fastTe

**Do some preprocessing.**

In [0]:
# Preprocess the data set
import pandas as pd
import csv
from io import StringIO
import re
from nltk.tokenize import TweetTokenizer
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#stop_words = stopwords.words('english')
# add other unnecessary tokens to stopping words
stops = ["you are", "you'r", "i ", "am", "on", "we", "we r", "they","i'm", "...", "->", ",", ";"]

def preprocess_tweets(tweets):
    tknzr = TweetTokenizer()
    for i in range(len(tweets)):
        # Remove hashtags, mentions, links and numbers from tweet. 
        tweets[i] = re.sub(r"@(\w+)", ' ', tweets[i], flags=re.MULTILINE)
        tweets[i] = re.sub(r"#(\w+)", ' ', tweets[i], flags=re.MULTILINE)
        tweets[i] = re.sub(r"http\S+", " ", tweets[i], flags=re.MULTILINE)
        tweets[i] = re.sub(r'[0-9]+', '', tweets[i], flags=re.MULTILINE)
        # remove stop words and single char tokens 
        tweets[i] = [token for token in tknzr.tokenize(tweets[i].lower()) if token not in stops and len(token) > 1]
        # flatten to a single string
        tweets[i] = ' '.join(tweets[i])

Format the inputs for FastText model.
```
Add __label__ prefix.
```

In [9]:
%cd ..
tweets = pd.read_csv('Tweets.csv')
tweets_text = tweets["text"].to_numpy()
preprocess_tweets(tweets_text)
#print(tweets.loc[0,'text'])
col = ['airline_sentiment', 'text']
tweets = tweets[col]
tweets = tweets[pd.notnull(tweets['text'])]
tweets.columns = ['airline_sentiment', 'text']
tweets.head()



tweets['airline_sentiment']=['__label__'+s.replace(' or ', '$').replace(', or ','$').replace(',','$').replace(' ','_').replace(',','__label__').replace('$$','$').replace('$',' __label__').replace('___','__') for s in tweets['airline_sentiment']]
tweets['airline_sentiment']

tweets['text']= tweets['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

tweets.to_csv(r'tweets.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")


/content


**Split the dataset.**

In [0]:
!head -n 10998 tweets.txt > tweets.train.txt
!tail -n 3642 tweets.txt > tweets.valid.txt

**Train the model.**

In [11]:
!./fastText-0.9.1/fasttext supervised -input tweets.train.txt -output model_tweet_predict -epoch 20 -wordNgrams 1

Read 0M words
Number of words:  9304
Number of labels: 3
Progress: 100.0% words/sec/thread:  496203 lr:  0.000000 loss:  0.337602 ETA:   0h 0m


**Get precision and recall values.**

In [12]:
!./fastText-0.9.1/fasttext test model_tweet_predict.bin tweets.valid.txt

N	3642
P@1	0.803
R@1	0.803


**Test the model for your inputs.**

In [13]:
!./fastText-0.9.1/fasttext predict model_tweet_predict.bin -

learn DL in 2 weeks!!
__label__neutral
disappointed by customer service of US Airline.  they are constantly canceling flights.
__label__negative
^C
